In [1]:
###################################################################
######  initial setup                                       #######
###################################################################
import pandas as pd
import numpy as np
import selenium
import random
import time
from numpy import heaviside
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy, ProxyType


In [2]:
###################################################################
######  pull previous data   \       setup                  #######
###################################################################
#setup variables
location_city = 'Charlotte'
location_state = 'NC'

#imported data
zip_code_data = pd.read_excel('uszips.xlsx')
zip_code_data = zip_code_data.query('city == @location_city').query('state_id == @location_state')

#empty datasets
house_links_list = []
checked_zips = []
checked_zips1 = []
checked_houses = []
house_df = pd.DataFrame()
schools_df = pd.DataFrame()
transactions_df = pd.DataFrame()

#previous data
try:
    house_links_list = list(np.genfromtxt("house_links.csv", dtype=str))
except:
    print('caution - cannot find the house_links csv - did you delete it?')
    pass
try:
    checked_zips = list(np.genfromtxt("checked_zips.csv", dtype=str))
except:
    print('caution - cannot find the checked_zips csv - did you delete it?')
    pass
try:
    checked_zips = list(np.genfromtxt("checked_zips1.csv", dtype=str))
except:
    print('caution - cannot find the checked_zips1 csv - did you delete it?')
    pass
try:
    checked_houses = list(np.genfromtxt("checked_houses.csv", dtype=str))
except:
    print('caution - cannot find the checked_houses csv - did you delete it?')
    pass
try: 
    house_df = pd.read_excel('df_house.xlsx')
except:
    print('caution - cannot find the df_house xlsx - did you delete it?')
    pass
try: 
    schools_df = pd.read_excel('df_schools.xlsx')
except:
    print('caution - cannot find the df_schools xlsx - did you delete it?')
    pass
try: 
    transactions_df = pd.read_excel('df_transactions.xlsx')
except:
    print('caution - cannot find the df_transactions xlsx - did you delete it?')
    pass


C:\Users\benle\AppData\Local\Temp/ipykernel_20772/81706297.py:23: UserWarning: genfromtxt: Empty input file: "house_links.csv"
  house_links_list = list(np.genfromtxt("house_links.csv", dtype=str))
C:\Users\benle\AppData\Local\Temp/ipykernel_20772/81706297.py:28: UserWarning: genfromtxt: Empty input file: "checked_zips.csv"
  checked_zips = list(np.genfromtxt("checked_zips.csv", dtype=str))
C:\Users\benle\AppData\Local\Temp/ipykernel_20772/81706297.py:33: UserWarning: genfromtxt: Empty input file: "checked_zips1.csv"
  checked_zips = list(np.genfromtxt("checked_zips1.csv", dtype=str))
C:\Users\benle\AppData\Local\Temp/ipykernel_20772/81706297.py:38: UserWarning: genfromtxt: Empty input file: "checked_houses.csv"
  checked_houses = list(np.genfromtxt("checked_houses.csv", dtype=str))


In [7]:
###################################################################
######  selenium driver                                     #######
###################################################################
#disable images
chrome_options = webdriver.ChromeOptions()
#prefs = {"profile.managed_default_content_settings.images": 2}
#chrome_options.add_experimental_option("prefs", prefs)
#Start driver
driver = webdriver.Chrome(executable_path='Users\benle\Downloads\chromedriver_win32 (1).zip')
url = 'https://www.redfin.com/'
driver.get(url)


C:\Users\benle\AppData\Local\Temp/ipykernel_20772/2868275809.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='Users\benle\Downloads\chromedriver_win32 (1).zip')


In [8]:
#Get Houses in city by zip code
for zip in zip_code_data['zip']:
    if zip not in checked_zips:
        url = f'https://www.redfin.com/zipcode/{zip}'.format(zip)
        driver.get(url)
        try:
            next_button = driver.find_element_by_xpath('//button[@class="clickable buttonControl button-text"]')
        except:
            next_button = ''
        time.sleep(5)
        loop = True
        while loop == True:
            try:
                houses = driver.find_elements_by_xpath('/html/body/div[1]/div[9]/div[2]/div[1]/div/div/div/div/div/div/div/div/div[1]/div[1]/div/div/a')
                for house in houses: 
                    if house.get_attribute('href') not in house_links_list:
                        house_links_list.append(house.get_attribute('href'))
                time.sleep(5)
                if next_button == '':
                    loop = False
                else:
                    next_button.click()
            except:
                loop = False
        checked_zips.append(zip)

In [22]:
#redo 'Get Houses in city by zip code', but remove outline to try and capture more houses
for zip in zip_code_data['zip']:
    if zip not in checked_zips1:
        url = f'https://www.redfin.com/zipcode/{zip}'.format(zip)
        driver.get(url)
        try:
            outline = driver.find_element_by_xpath('/html/body/div[1]/div[9]/div[1]/div[3]/div/div[2]/div[1]/div/div/button')
            outline.click()
        except:
            pass
        time.sleep(5)
        try:
            next_button = driver.find_element_by_xpath('//button[@class="clickable buttonControl button-text"]')
            print('try')
            print(next_button)
        except:
            next_button = ''
            print("except")
            print(next_button)
        loop = True
        while loop == True:
            try:
                houses = driver.find_elements_by_xpath('/html/body/div[1]/div[9]/div[2]/div[1]/div/div/div/div/div/div/div/div/div[1]/div[1]/div/div/a')
                for house in houses: 
                    if house.get_attribute('href') not in house_links_list:
                        house_links_list.append(house.get_attribute('href'))
                time.sleep(5)
                if next_button == '':
                    loop = False
                else:
                    next_button.click()
            except:
                loop = False
        checked_zips1.append(zip)

In [16]:
for url in house_links_list:
    if url not in checked_houses:
        driver.get(url)
        time.sleep(7)
        
        #header
        try:
            address = driver.find_element_by_xpath('/html/body/div[1]/div[12]/div[2]/div[1]/div/div[1]/div/div/div/div[1]/header/h1/div[1]')
            address_extended = driver.find_element_by_xpath('/html/body/div[1]/div[12]/div[2]/div[1]/div/div[1]/div/div/div/div[1]/header/h1/div[2]')
            beds = driver.find_element_by_xpath('/html/body/div[1]/div[12]/div[2]/div[1]/div/div[1]/div/div/div/div[1]/div/div[2]/div')
            baths = driver.find_element_by_xpath('/html/body/div[1]/div[12]/div[2]/div[1]/div/div[1]/div/div/div/div[1]/div/div[3]/div')
            sqft = driver.find_element_by_xpath('/html/body/div[1]/div[12]/div[2]/div[1]/div/div[1]/div/div/div/div[1]/div/div[4]/span')
            price = driver.find_element_by_xpath('//*[@id="content"]/div[12]/div[2]/div[1]/div/div[1]/div/div/div/div[1]/div/div[1]/div')
            address = address.text
            address_extended = address_extended.text
            beds = beds.text
            baths = baths.text
            sqft = sqft.text
            price = price.text
        except:
            address = ''
            address_extended = ''
            beds = ''
            baths = ''
            sqft = ''
            price = ''

        #description
        try:
            continue_reading = driver.find_element_by_xpath('/html/body/div[1]/div[12]/div[2]/div[7]/section/div/div/div/div/div/div[1]/div/div[2]/div/span')
            continue_reading.click()
        except:
            pass
        try:
            description = driver.find_element_by_xpath('//*[@id="marketing-remarks-scroll"]')
            description = description.text
        except:
            description = ''
        try:
            updated = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div[2]/div[2]/div[1]')
            updated = updated.text
        except:
            updated = ''
        try:
            header_for_yoy = driver.find_element_by_xpath('//*[@id="marketInsights-collapsible"]/div[1]/div/div/div/h2')
            driver.execute_script("arguments[0].scrollIntoView();", header_for_yoy)
            time.sleep(1)
        except:
            pass

        #home facts
        try:
            property_type = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div/div[contains(string(),"Property Type")]/span[2]')
            property_type = property_type.text
        except:
            property_type = ''

        try:
            style = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div/div[contains(string(),"Style")]/span[2]')
            style = style.text
        except:
            style = ''

        try:
            lot_size = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div/div[contains(string(),"Lot Size")]/span[2]')
            lot_size = lot_size.text
        except:
            lot_size = ''

        try:
            time_on_redfin = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div/div[contains(string(),"Time on Redfin")]/span[2]')
            time_on_redfin = time_on_redfin.text
        except:
            time_on_redfin = ''

        try:
            year_built = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div/div[contains(string(),"Year Built")]/span[2]')
            year_built = year_built.text
        except:
            year_built = ''

        try:
            community = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div/div[contains(string(),"Community")]/span[2]')
            community = community.text
        except:
            community = ''

        #price insights
        try:
            price_per_sqft = driver.find_element_by_xpath('//*[@id="house-info"]/div/div/div/div[contains(string(),"Price/Sq.Ft.")]/span[2]')
            price_per_sqft = price_per_sqft.text
        except:
            price_per_sqft = ''

        #property details - will extract this data with regex later:
        property_details = driver.find_element_by_xpath('//*[@id="propertyDetails-collapsible"]/div[2]/div/div/div[1]')

        #around this home
        try:
            walk_score = driver.find_element_by_xpath('//div[@class="transport-icon-and-percentage walkscore"]/div[@data-rf-test-name="ws-percentage"]/span[1]')
            walk_score = walk_score.text
            transit_score = driver.find_element_by_xpath('//div[@class="transport-icon-and-percentage transitscore"]/div[@data-rf-test-name="ws-percentage"]/span[1]')
            transit_score = transit_score.text
            bike_score = driver.find_element_by_xpath('//div[@class="transport-icon-and-percentage bikescore"]/div[@data-rf-test-name="ws-percentage"]/span[1]')
            bike_score = bike_score.text
        except:
            walk_score = ''
            transit_score = ''
            bike_score = ''

        #activity
        try:
            view_count = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[1]/div/div[2]/div/span[1]')
            current_favorite = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[2]/div/div[2]/div[1]/span[1]')
            alltime_favorite = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[2]/div/div[2]/div[2]/span[1]')
            current_favorite = current_favorite.text
            current_xout = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[3]/div/div[2]/div[1]/span[1]')
            alltime_xout = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[3]/div/div[2]/div[2]/span[1]')
            current_xout = current_xout.text
        except:
            view_count = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[1]/div/div[2]/div/span[1]')
            alltime_favorite  = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[2]/div/div[2]/div[1]/span[1]')
            current_favorite = ''
            alltime_xout = driver.find_element_by_xpath('//table[@class="activityStatsTable"]/tbody/tr/td[3]/div/div[2]/div[1]/span[1]')
            current_xout = ''

        #public facts
        try:
            stories = driver.find_element_by_xpath('//*[@id="basicInfo"]/div[2]/div[1]/div[contains(string(),"Stories")]/div')
            stories = stories.text
        except:
            stories = ''

        try:
            lot = driver.find_element_by_xpath('//*[@id="basicInfo"]/div[2]/div[1]/div[contains(string(),"Lot Size")]/div')
            lot = lot.text
        except:
            lot = ''

        try:
            renovated_year = driver.find_element_by_xpath('//*[@id="basicInfo"]/div[2]/div[1]/div[contains(string(),"Year Renovated")]/div')
            renovated_year = renovated_year.text
        except:
            renovated_year = ''

        #market insights
        try:
            median_list_30d = driver.find_element_by_xpath('//li[@aria-hidden="false"]/div/div/div[@class="MarketInsightsRegionCard--cardMetrics row"]/div[1]/div[1]').text
            median_days_on_market_30d = driver.find_element_by_xpath('//li[@aria-hidden="false"]/div/div/div[@class="MarketInsightsRegionCard--cardMetrics row"]/div[2]/div[1]').text
            other_listings_nearby_30d = driver.find_element_by_xpath('//li[@aria-hidden="false"]/div/div/div[@class="MarketInsightsRegionCard--cardMetrics row"]/div[3]/div[1]').text
            median_price_per_sqft_30d = driver.find_element_by_xpath('//li[@aria-hidden="false"]/div/div/div[@class="MarketInsightsRegionCard--cardMetrics row"]/div[4]/div[1]').text
            median_sale_to_list_30d = driver.find_element_by_xpath('//li[@aria-hidden="false"]/div/div/div[@class="MarketInsightsRegionCard--cardMetrics row"]/div[5]/div[1]').text
            sold_homes_nearby_30d = driver.find_element_by_xpath('//li[@aria-hidden="false"]/div/div/div[@class="MarketInsightsRegionCard--cardMetrics row"]/div[6]/div[1]').text
        except:
            median_list_30d = ''
            median_days_on_market_30d = ''
            other_listings_nearby_30d = ''
            median_price_per_sqft_30d = ''
            median_sale_to_list_30d = ''
            sold_homes_nearby_30d = ''
            pass

        try:
            header_for_yoy = driver.find_element_by_xpath('//*[@id="marketInsights-collapsible"]/div[1]/div/div/div/h2')
            driver.execute_script("arguments[0].scrollIntoView();", header_for_yoy)
            time.sleep(1)
        except:
            pass
        try:
            median_sale_price_yoy = driver.find_element_by_xpath('//div[@class="ModeToggler dataTabs"]/button[1]/div/div[2]/div[1]')
            median_sale_price_yoy = median_sale_price_yoy.text
            median_sale_trend_yoy = driver.find_element_by_xpath('//div[@class="ModeToggler dataTabs"]/button[1]/div/div[2]/div[2]')
            median_sale_trend_yoy = median_sale_trend_yoy.text
            homes_sold_yoy = driver.find_element_by_xpath('//div[@class="ModeToggler dataTabs"]/button[2]/div/div[2]/div[1]')
            homes_sold_yoy = homes_sold_yoy.text
            homes_sold_trend_yoy = driver.find_element_by_xpath('//div[@class="ModeToggler dataTabs"]/button[2]/div/div[2]/div[2]')
            homes_sold_trend_yoy = homes_sold_trend_yoy.text
            days_on_market_yoy = driver.find_element_by_xpath('//div[@class="ModeToggler dataTabs"]/button[3]/div/div[2]/div[1]')
            days_on_market_yoy = days_on_market_yoy.text
            days_on_market_trend_yoy = driver.find_element_by_xpath('//div[@class="ModeToggler dataTabs"]/button[3]/div/div[2]/div[2]')
            days_on_market_trend_yoy = days_on_market_trend_yoy.text
        except:
            median_sale_price_yoy = ''
            median_sale_trend_yoy = ''
            homes_sold_yoy = ''
            homes_sold_trend_yoy = ''
            days_on_market_yoy = ''
            days_on_market_trend_yoy = ''

        #market Competition
        try:
            area_redfin_compete_score = driver.find_element_by_xpath('//div[@class="scoreTM"]/div[1]').text
            area_facts = driver.find_element_by_xpath('//*[@id="marketInsights-collapsible"]/div[2]/div/div/div[@class="scoreDetails"]').text
        except:
            area_redfin_compete_score = ''
            area_facts = ''

        temp = {
            'url_key':driver.current_url,
            'address':address,
            'address_extended':address_extended,
            'beds':beds,
            'baths':baths,
            'sqft':sqft,
            'price':price,
            'lot':lot,
            'stories':stories,
            'description':description,
            'updated':updated,
            'property_type':property_type,
            'style':style,
            'lot_size':lot_size,
            'time_on_redfin':time_on_redfin,
            'year_built':year_built,
            'price_per_sqft':price_per_sqft,
            'property_details':property_details.text,
            'walk_score':walk_score,
            'transit_score':transit_score,
            'bike_score':bike_score,
            'view_count':view_count.text,
            'current_favorite':current_favorite,
            'alltime_favorite':alltime_favorite.text,
            'current_xout':current_xout,
            'alltime_xout':alltime_xout.text,
            'renovated_year':renovated_year,
            'median_list_30d':median_list_30d,
            'median_days_on_market_30d':median_days_on_market_30d,
            'other_listings_nearby_30d':other_listings_nearby_30d,
            'median_price_per_sqft_30d':median_price_per_sqft_30d,
            'median_sale_to_list_30d':median_sale_to_list_30d,
            'sold_homes_nearby_30d':sold_homes_nearby_30d,
            'median_sale_price_yoy':median_sale_price_yoy,
            'median_sale_trend_yoy':median_sale_trend_yoy,
            'homes_sold_yoy':homes_sold_yoy,
            'homes_sold_trend_yoy':homes_sold_trend_yoy,
            'days_on_market_yoy':days_on_market_yoy,
            'days_on_market_trend_yoy':days_on_market_trend_yoy,
            'area_redfin_compete_score':area_redfin_compete_score,
            'area_facts':area_facts
        }
        house_df = house_df.append(temp, ignore_index=True)

        #school scrape
        schools = driver.find_elements_by_xpath('//div[@class="schools-content"]/div/div/div/div/div[2]/div[1]/div[1]')
        overall_rating = driver.find_elements_by_xpath('//div[@class="schools-content"]/div/div/div/div/div[1]/div[1]/span[1]')
        student_count = driver.find_elements_by_xpath('//div[@class="schools-content"]/div/div/div/div/div[2]/div[2]/div[1]')
        school_distance = driver.find_elements_by_xpath('//div[@class="schools-content"]/div/div/div/div/div[2]/div[3]/div[1]')
        review_count = driver.find_elements_by_xpath('//div[@class="schools-content"]/div/div/div/div/div[2]/div[4]/div[2]')
        review_rating = driver.find_elements_by_xpath('//div[@class="schools-content"]/div/div/div/div/div[2]/div[4]/div[1]/div[1]/div[1]')
        for a in range(len(schools)):
            temp_school = {
                'url_key':driver.current_url,
                'schools':schools[a].text,
                'overall_rating':overall_rating[a].text,
                'student_count':student_count[a].text,
                'school_distance':school_distance[a].text,
                'review_count':review_count[a].text,
                'review_rating':review_rating[a].get_attribute('aria-label')
            }
            schools_df = schools_df.append(temp_school, ignore_index=True)

        #sales history scrape
        try:
            see_all_history = driver.find_element_by_xpath('//*[@id="propertyHistory-expandable-segment"]/div[2]/div/span')
            see_all_history.click()
        except:
            pass
        transaction_dates = driver.find_elements_by_xpath('//div[@id="property-history-transition-node"]/div/div/div/div/div/div/div/div/div/div[1]/p[1]')
        transaction_type = driver.find_elements_by_xpath('//div[@id="property-history-transition-node"]/div/div/div/div/div/div/div/div/div/div[2]/div[1][not(@class="sectionBottomLink")]')
        transaction_amount = driver.find_elements_by_xpath('//div[@id="property-history-transition-node"]/div/div/div/div/div/div/div/div/div/div[3]/div[1]')

        for a in range(len(transaction_dates)):
            temp_transactions = {
                'url_key':driver.current_url,
                'transaction_dates':transaction_dates[a].text,
                'transaction_type':transaction_type[a].text,
                'transaction_amount':transaction_amount[a].text
            }
            transactions_df = transactions_df.append(temp_transactions, ignore_index=True)
            
        #gather more houses

        nearby_homes = driver.find_elements_by_xpath('//*[@id="nearbySimilarHomesCarousel"]/li/div/div/div[1]/a')
        for homes in nearby_homes:
            if homes.get_attribute('href') not in house_links_list:
                house_links_list.append(homes.get_attribute('href'))

        recent_sales = driver.find_elements_by_xpath('//*[@id="redfin-estimate"]/div[2]/div/div/div/div[1]/div/div/ul/li/div/div/div[1]/div[1]/div/div/a')
        for homes in recent_sales:
            if homes.get_attribute('href') not in house_links_list:
                house_links_list.append(homes.get_attribute('href'))

        recommended_houses = driver.find_elements_by_xpath('//*[@id="lsis-recommendeds"]/div[2]/div/div/div/div[1]/div[1]/div/div/a')
        for homes in recommended_houses:
            if homes.get_attribute('href') not in house_links_list:
                house_links_list.append(homes.get_attribute('href'))

        similiar_homes = driver.find_elements_by_xpath('//*[@id="lsis-listings"]/div[2]/div/div/div/div[1]/div[1]/div/div/a')
        for homes in similiar_homes:
            if homes.get_attribute('href') not in house_links_list:
                house_links_list.append(homes.get_attribute('href'))

        nearby_sales = driver.find_elements_by_xpath('//*[@id="lsis-solds"]/div[2]/div/div/div/div[1]/div[1]/div/div/a')
        for homes in nearby_sales:
            if homes.get_attribute('href') not in house_links_list:
                house_links_list.append(homes.get_attribute('href'))

        #check off link
        checked_houses.append(url)



In [10]:
###################################################################
######  save data                                           #######
###################################################################

np.savetxt("house_links.csv", 
           house_links_list,
           delimiter =", ", 
           fmt ='% s')

np.savetxt("checked_zips.csv", 
           checked_zips,
           delimiter =", ", 
           fmt ='% s')

np.savetxt("checked_zips1.csv", 
           checked_zips1,
           delimiter =", ", 
           fmt ='% s')

np.savetxt("checked_houses.csv", 
           checked_houses,
           delimiter =", ", 
           fmt ='% s')

house_df.to_excel('df_house.xlsx', index=False)
schools_df.to_excel('df_schools.xlsx', index=False)
transactions_df.to_excel('df_transactions.xlsx', index=False)

In [11]:
house_df

,url_key,address,address_extended,beds,baths,sqft,price,lot,stories,description,...,median_sale_to_list_30d,sold_homes_nearby_30d,median_sale_price_yoy,median_sale_trend_yoy,homes_sold_yoy,homes_sold_trend_yoy,days_on_market_yoy,days_on_market_trend_yoy,area_redfin_compete_score,area_facts
